# 지하철 시간대별 승하차 비율 분석을 통한 주거지/업무지 분포 추정 보고서

## 1. 프로젝트 개요
- 분석 목적 : 지하철역별 출퇴근 시간 승하차 패턴을 분석하여, 서울 시민의 주거지와 업무지의 공간적 분포를 추정하고, 이에 따른 교통 혼잡 개선 방향을 제시
- 데이터 출처 : 서울 열린데이터 광장
- 분석 대상 변수:
    - 07시-08시 승차인원, 07시-08시 하차인원
    - 08시-09시 승차인원, 08시-09시 하차인원
    - 17시-18시 승차인원, 17시-18시 하차인원
    - 18시-19시 승차인원, 18시-19시 하차인원
    - 호선명
    - 지하철역
    - 작업일자

In [5]:
import pandas as pd

df = pd.read_csv("./data/Seoul_Subways_time.csv", encoding='cp949')

# 기본 정보 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74277 entries, 0 to 74276
Data columns (total 52 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   사용월           74277 non-null  int64 
 1   호선명           74277 non-null  object
 2   지하철역          74277 non-null  object
 3   04시-05시 승차인원  74277 non-null  int64 
 4   04시-05시 하차인원  74277 non-null  int64 
 5   05시-06시 승차인원  74277 non-null  int64 
 6   05시-06시 하차인원  74277 non-null  int64 
 7   06시-07시 승차인원  74277 non-null  int64 
 8   06시-07시 하차인원  74277 non-null  int64 
 9   07시-08시 승차인원  74277 non-null  int64 
 10  07시-08시 하차인원  74277 non-null  int64 
 11  08시-09시 승차인원  74277 non-null  int64 
 12  08시-09시 하차인원  74277 non-null  int64 
 13  09시-10시 승차인원  74277 non-null  int64 
 14  09시-10시 하차인원  74277 non-null  int64 
 15  10시-11시 승차인원  74277 non-null  int64 
 16  10시-11시 하차인원  74277 non-null  int64 
 17  11시-12시 승차인원  74277 non-null  int64 
 18  11시-12시 하차인원  74277 non-null  int64 
 19  12시-

In [ ]:
# 데이터 샘플 보기
df.head()

,사용월,호선명,지하철역,04시-05시 승차인원,04시-05시 하차인원,05시-06시 승차인원,05시-06시 하차인원,06시-07시 승차인원,06시-07시 하차인원,07시-08시 승차인원,...,23시-24시 하차인원,00시-01시 승차인원,00시-01시 하차인원,01시-02시 승차인원,01시-02시 하차인원,02시-03시 승차인원,02시-03시 하차인원,03시-04시 승차인원,03시-04시 하차인원,작업일자
0,202505,1호선,동대문,695,16,11411,1920,9784,7385,16724,...,9851,903,3141,8,284,0,0,0,0,20250603
1,202505,1호선,동묘앞,148,3,2859,986,4018,5686,8874,...,4711,135,2029,3,178,0,0,0,0,20250603
2,202505,1호선,서울역,688,67,10783,10374,28788,65184,90177,...,32808,4299,8636,18,174,0,0,0,0,20250603
3,202505,1호선,시청,121,3,1938,5755,3754,27708,7542,...,3883,558,899,0,32,0,0,0,0,20250603
4,202505,1호선,신설동,414,32,9502,2362,10403,9078,22203,...,9707,444,3017,0,0,0,0,0,0,20250603


## 2. 데이터 전처리

In [18]:
missing_count = df.isnull().sum()
#결측치는 없는 것을 확인
missing_count[missing_count > 0]

Series([], dtype: int64)

In [11]:
# 필요없는 데이터 제거

# 출퇴근 시간대의 데이터만 사용
df_keep = ['사용월', '호선명', '지하철역', '07시-08시 승차인원', '07시-08시 하차인원', '08시-09시 승차인원', '08시-09시 하차인원',\
            '17시-18시 승차인원','17시-18시 하차인원','18시-19시 승차인원','18시-19시 하차인원', '작업일자']
subways_df = df[df_keep]
# 최근 5년간의 데이터만 사용
subways_df = subways_df[subways_df['사용월'] >= 202001]

In [19]:
#폰트 지정
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams['font.family'] = 'Hancom Gothic'
rcParams['axes.unicode_minus'] = False

## 3. 탐색적 데이터 분석 (EDA)

In [43]:
subways_df.groupby('지하철역')['07시-08시 승차인원'].describe().sort_values('mean',ascending=False).head(10)

,count,mean,std,min,25%,50%,75%,max
지하철역,,,,,,,,
구로디지털단지,65.0,131361.446154,11890.082594,99710.0,124041.00,133096.0,140436.0,151803.0
서울대입구(관악구청),65.0,119738.938462,10650.413786,91257.0,112846.00,121903.0,128349.0,138735.0
신림,102.0,118948.598039,87860.473969,352.0,4402.25,173535.0,186471.5,217412.0
쌍문,65.0,118533.430769,9686.885764,94029.0,111826.00,120597.0,126035.0,137926.0
까치산,65.0,116967.092308,10124.233108,91796.0,109227.00,119807.0,124148.0,136255.0
역곡,65.0,111747.661538,8940.719557,89543.0,104946.00,113340.0,118161.0,128743.0
화곡,65.0,109563.584615,9840.795031,85383.0,102161.00,111603.0,115986.0,128489.0
수유(강북구청),65.0,106764.738462,9324.142533,83692.0,100164.00,109041.0,113622.0,124584.0
송내,65.0,102359.353846,8878.366310,78803.0,96231.00,104314.0,109133.0,120100.0


In [46]:
subways_df.groupby('지하철역')['08시-09시 승차인원'].describe().sort_values('mean',ascending=False).head(10)

,count,mean,std,min,25%,50%,75%,max
지하철역,,,,,,,,
신림,102.0,178113.588235,131216.248670,576.0,7040.75,258908.5,278345.00,323248.0
구로디지털단지,65.0,175572.923077,13971.274589,138972.0,164557.00,179261.0,185427.00,201815.0
서울대입구(관악구청),65.0,169540.476923,12494.777861,137157.0,162572.00,172422.0,177042.00,196629.0
까치산,65.0,146391.507692,10806.037353,120806.0,139289.00,148213.0,154452.00,167412.0
화곡,65.0,139599.784615,11461.771840,112350.0,130794.00,142462.0,149208.00,161580.0
신대방,65.0,125445.876923,9909.807274,100366.0,118517.00,127476.0,132899.00,143556.0
낙성대,2.0,124638.500000,4806.204792,121240.0,122939.25,124638.5,126337.75,128037.0
쌍문,65.0,123711.076923,8898.033816,96253.0,117938.00,123122.0,131162.00,138634.0
낙성대(강감찬),63.0,119446.587302,9929.324418,95826.0,111530.00,122371.0,127148.50,136581.0


아침 출근시간을 7~8시, 8~9시로 나누어 승차하는 사람들이 많은 역을 상위 10위건만 보았을때<br>
신림, 구로디지털단지, 서울대입구(관악구청), 까치산, 화곡, 쌍문, 수유(강북구청)이 중복으로 나오는 것을 알 수 있다.<br>
이를 통해 이 지역에 거주하는 사람들이 많은것으로 예측해 볼 수 있다.

,count,mean,std,min,25%,50%,75%,max
지하철역,,,,,,,,
신림,102.0,178113.588235,131216.248670,576.0,7040.75,258908.5,278345.00,323248.0
구로디지털단지,65.0,175572.923077,13971.274589,138972.0,164557.00,179261.0,185427.00,201815.0
서울대입구(관악구청),65.0,169540.476923,12494.777861,137157.0,162572.00,172422.0,177042.00,196629.0
까치산,65.0,146391.507692,10806.037353,120806.0,139289.00,148213.0,154452.00,167412.0
화곡,65.0,139599.784615,11461.771840,112350.0,130794.00,142462.0,149208.00,161580.0
신대방,65.0,125445.876923,9909.807274,100366.0,118517.00,127476.0,132899.00,143556.0
낙성대,2.0,124638.500000,4806.204792,121240.0,122939.25,124638.5,126337.75,128037.0
쌍문,65.0,123711.076923,8898.033816,96253.0,117938.00,123122.0,131162.00,138634.0
낙성대(강감찬),63.0,119446.587302,9929.324418,95826.0,111530.00,122371.0,127148.50,136581.0
